## Can hexapods explain guider drifts?

Craig Lage  02-Dec-25

In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
from astropy.time import Time, TimeDelta
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.simonyi.mountData import MountData
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors, plotMountErrors, \
        getAltAzOverPeriod, getAzElRotHexDataForExposure
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from lsst.obs.lsst import LsstCam
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord, angular_separation
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData


In [ ]:
def HexapodDrifts(mountData: MountData) -> tuple[float, float]:
    """Calculate the image drift associated with the hexapod motions.
    Parameters
    ----------
    mountData : MountData
        The EFD data associated with the exposure
    Returns
    -------
    tuple[float, float]
        The image motions associated with the CamHex and M2Hex motions.
        (AltDrift, AzDrift)
    """
    # The below image motion coefficients were calculated
    # with a Batoid simulation by Josh Meyers
    camHexXY = 1.00  # microns(image) / micron(hexapod)
    camHexUV = 4.92  # microns(image) / arcsecond(hexapod)
    m2HexXY = 1.13  # microns(image) / micron(hexapod)
    m2HexUV = 37.26  # microns(image) / arcsecond(hexapod)

    # Convert these to image impact in arcseconds
    # The 10.0 is microns / pixel
    pixelScale = 0.2  # arcseconds / pixel - find this elsewhere?
    camHexXY = camHexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
    camHexUV = camHexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
    # Convention here is [Alt, Az] at rotator=0
    camCoefs = np.array([[camHexXY,0.0], [0.0, -camHexXY], \
                        [0.0,0.0], [0.0, camHexUV], [camHexUV, 0.0]])
    m2HexXY = m2HexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
    m2HexUV = m2HexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
    # Convention here is [Alt, Az] at rotator=0
    m2Coefs = np.array([[m2HexXY,0.0], [0.0, -m2HexXY], \
                        [0.0,0.0], [m2HexUV, 0.0], [0.0, m2HexUV]])

    drift = (0.0, 0.0)

    # For each hexapod motion, fit a line and determine the slope
    camhex = mountData.camhexData
    times = camhex['private_kafkaStamp']
    times -= times.iloc[0]
    for i in [0, 1, 3, 4]: # Z has no impact
        dat = camhex[f'position{i}']
        fit = np.polyfit(times, dat, 1)
        change = fit[0] * times.iloc[-1]  * camCoefs[i]
        #print(f"Cam, {i}, {fit[0]}, {change}")
        drift += change

    m2hex = mountData.m2hexData
    times = m2hex['private_kafkaStamp']
    times -= times.iloc[0]
    for i in [0, 1, 3, 4]: # Z has no impact
        dat = m2hex[f'position{i}']
        fit = np.polyfit(times, dat, 1)
        change = fit[0] * times.iloc[-1] * m2Coefs[i]
        #print(f"M2, {i}, {fit[0]}, {change}")
        drift += change

    # Now rotate it into the AltAz coords.
    rot = np.median(mountData.rotationData['actualPosition'].values)
    theta = np.pi * rot / 180.0
    R = [[ np.cos(theta), -np.sin(theta)],
     [ np.sin(theta),  np.cos(theta)]]
    drift = tuple(np.dot(R, drift)) #(Alt, Az)
    return drift

In [ ]:
client = makeEfdClient()
butler = Butler('/repo/embargo', collections=["LSSTCam/raw/all", "LSSTCam/calib", \
                                        "LSSTCam/runs/nightlyValidation"])
instrument = 'LSSTCam'
camera = LsstCam.getCamera()

In [ ]:
expId = 2025120200023
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)
(mountErrors, mountData) = calculateMountErrors(expRecord, client)
np.median(mountData.rotationData['actualPosition'].values)

In [ ]:
drift = HexapodDrifts(mountData)
drift

In [ ]:
def AltAzforExposure(expId, detector):
    wavelengths = {'u':3671, 'g':4827, 'r':6223, 'i':7546, 'z':8691, 'y':9712}
    bbox = detector.getBBox()
    
    rawExp = butler.get('raw', detector=detector.getId(), exposure=expId, instrument=instrument)
    md = rawExp.getMetadata()
    
    calExp = butler.get('preliminary_visit_image', detector=detector.getId(), visit=expId, instrument=instrument)
    cWcs = calExp.getWcs()
    calExpSkyCenter = cWcs.pixelToSky(Point2D(bbox.centerX, bbox.centerY))
    true_ra = calExpSkyCenter.getRa().asDegrees()
    true_dec = calExpSkyCenter.getDec().asDegrees()
    skyLocation = SkyCoord(true_ra*u.deg, true_dec*u.deg)
    filter = md['FILTBAND']
    pressure = md['PRESSURE'] * u.pascal
    temperature = md['AIRTEMP'] * u.Celsius
    hum = md['HUMIDITY']
    time = Time((md['MJD-BEG'] + md['MJD-END']) / 2.0, format='mjd', scale='tai')
    wl = wavelengths[filter] * u.angstrom
    altAz = AltAz(obstime=time, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    obsAltAz = skyLocation.transform_to(altAz)
    return obsAltAz

In [ ]:
expId = 2025120200023
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)
(mountErrors, mountData) = calculateMountErrors(expRecord, client)
np.median(mountData.rotationData['actualPosition'].values)

In [ ]:
# Center CCD
obsAltAz = AltAzforExposure(expId, camera['R22_S11'])
print(obsAltAz.alt.deg, obsAltAz.az.deg)

In [ ]:
# +Y from center CCD gives positive change in Az
obsAltAz = AltAzforExposure(expId, camera['R22_S21'])
print(obsAltAz.alt.deg, obsAltAz.az.deg)

In [ ]:
# +X from center CCD gives negative change in Alt
obsAltAz = AltAzforExposure(expId, camera['R22_S12'])
print(obsAltAz.alt.deg, obsAltAz.az.deg)